#### Образовательная платформа: SkillFactory
#### Специализация: Data Science
#### Группа: DST-17
### Юнит 6. Проект 5: "Выбираем автомобиль правильно"
##### Выполнил: Владимир Юшманов

<img src="https://raw.githubusercontent.com/vyushmanov/skillfactory_rds/master/module_6/choice_the_car.jpg"/>

# Прогнозирование стоимости автомобиля по характеристикам


In [ ]:
import re
import sys
import numpy as np
import pandas as pd
import pandas.api.types as at
from tqdm import tqdm
from datetime import timedelta, datetime, date

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

import myfunction as mf

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 123

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 9
DIR_TRAIN  = '../input/autoru/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# 1. Данные

In [ ]:
!ls '../input'

train = pd.read_csv(DIR_TRAIN+'all_auto_ru_2021-01-08.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

pd.set_option('display.max_columns', None)
display(train.head(2))
display(test.head(2))

## 1.1. Исследование соответствия тренировочного и тестового наборов данных

In [ ]:
# Введем признак разделения на тренировочную и тестовую выбороки - sample
train['sample'] = 1
test['sample'] = 0

# Оставим в train только актуальные предложения (удалим скрытые объявления)
df_train = train[train['hidden'] == False]
df_test = test

In [ ]:
#  у участников найдена функция по сравнению дата-сетов. Немного преобразовал для удобства восприятия
def check_df_before_merg(d_df1,d_df2):
    
    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['feature_train'] = list_of_names1
    temp_dict['type_train'] = d_df1.dtypes
    temp_dict['sample_train'] = d_df1.loc[5].values
    temp_dict['# unique_train'] = d_df1.nunique().values
    temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['feature_test'] = list_of_names2
    temp_dict2['type_test'] = d_df2.dtypes
    temp_dict2['sample_test'] = d_df2.loc[5].values
    temp_dict2['# unique_test'] = d_df2.nunique().values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
    temp_insert = pd.DataFrame(columns=['< - >'])
    
    temp_df = pd.concat([temp_df1,temp_insert, temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    temp_df['< - >'] = '| - |'
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []
    temp_list5  = []

    for i in range(len(temp_df)):
        if str(temp_df3['type_train'][i]) != str(temp_df3['type_test'][i]):
            temp_list.append(temp_df3['feature_train'][i])
            temp_list2.append(temp_df3['feature_test'][i])
            temp_list3.append(str(temp_df3['type_train'][i]) + ' != ' + str(temp_df3['type_test'][i]))
            temp_list4.append(i)
        if temp_df3['# unique_test'][i]>0 and temp_df3['# unique_train'][i]/temp_df3['# unique_test'][i] > 2:
            temp_list5.append(i)
            
    temp_dict3['index']= temp_list4
    temp_dict3['feature_train']= temp_list
    temp_dict3['не совпадают типы'] = temp_list3
    temp_dict3['feature_test']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'Резюме:\n 1. Не совпали типы в:= {len(temp_df4)} столбцах\n')
    print(f'2. Уникальные значения различаются в:= {len(temp_list5)} столбцах {temp_list5}')
    display(temp_df4)

# После нескольких запусков выявили список столбцов, не полезных в дальнейшей работе
# Это признаки, имеющие единственное значение, признаки не имеющие пары в обучающем и тестовом датасетах, признаки, 
# содержащие консолидированную информацию, уже сохраненную в других признаках

drop_cols = ['car_url', 'sell_id', 'image', 'hidden', 'parsing_unixtime', 'Состояние']

for col in drop_cols:
    if col in list(df_train.columns):
        df_train.drop(columns=[col], inplace=True)
    if col in list(df_test.columns):
        df_test.drop(columns=[col], inplace=True)
    
check_df_before_merg(df_train, df_test)

## 1.2. Стандартизация данных
Приведение форматов и значений признаков тренировочной выборки к виду, принятому в тестовой выборке

### 1.2.1. bodyType
Произведем укрупнение признаков в тренировочных данных до уровня, принятого в тестовой выборке

In [ ]:
df_test['body_type'] = [str(x).lower().replace('.', '') for x in df_test['bodyType']]
df_train['body_type'] = [str(x).lower() for x in df_train['bodyType']]

# Заменим длинные типы в train'е на обобщенные значения
body_type_list = list(df_test['body_type'].unique())
def get_perf_type(x, body_type_list):
    for t in body_type_list:
        if t in x:
            return t
        else: continue
    else: return '0'
    
df_train['body_type'] = df_train['body_type'].apply(lambda x: get_perf_type(x, body_type_list))

# Удалим 7 предложений, для которых не нашлось соответствия типа кузова
df_train = df_train[df_train['body_type']!='0']
df_train.drop(columns=['bodyType'], inplace=True)
df_test.drop(columns=['bodyType'], inplace=True)

### 1.2.2. 

Тренировочный датасет содержит 36 марок. В тестовой выборке только 12: 'SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'
### 1.2.3. color
Приведем обозначение цвета к 

In [ ]:
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}
df_train['color'] = df_train['color'].map(dict_color)


### 1.2.4. complectation_dict
Вероятность использования признака мала. Его содержание повторяет другие признаки.
### 1.2.5. description
Описание технического состояния ТС, выполненное продавцом. Без применения анализа текстов признак не применим.
### 1.2.6. engineDisplacement
Ттребуется удаление символов 'LTR' в тестовой выборке.

In [ ]:
df_test['engineDisplacement'] = df_test['engineDisplacement'].apply(lambda x: str(x).replace('LTR', ''))

### 1.2.7. enginePower

In [ ]:
pattern = re.compile('^(\d+)\.*')
df_test['enginePower'] = df_test['enginePower'].apply(lambda x: int(str(pattern.findall(x))[2:-2]))
df_train['enginePower'] = df_train['enginePower'].astype('int')

### 1.2.8. equipment_dict
Пока оставим в качестве резервного источника информации. Поименение в модели без обработки невозможно.
### 1.2.9. fuelType
Преобразований не требуется
### 1.2.10. mileage
Преобразований не требуется
### 1.2.11. modelDate
Заполним пропуски значением 0 и поправим формат в тренировочных данных

In [ ]:
df_train['modelDate'] = df_train['modelDate'].fillna(0).astype('int')

### 1.2.12. model_name
Не требует преобразований
### 1.2.13. name
Резервный источник данных
### 1.2.14. numberOfDoors
Заполним пропуски значением 0 и поправим формат в тренировочных данных

In [ ]:
df_train['numberOfDoors'] = df_train['numberOfDoors'].fillna(0).astype('int')

### 1.2.15. priceCurrency

In [ ]:
df_train['priceCurrency'] = df_train['priceCurrency'].map({'RUR':'RUB'})

### 1.2.16. productionDate
Не требует преобразований
### 1.2.17. super_gen
Пока оставим в качестве резервного источника информации. Поименение в модели без обработки невозможно.
### 1.2.18. vehicleConfiguration
Пока оставим в качестве резервного источника информации. Поименение в модели без обработки невозможно.
### 1.2.19. vehicleTransmission


In [ ]:
df_train['vehicleTransmission'] = df_train['vehicleTransmission'].map({'AUTOMATIC':'автоматическая', 'MECHANICAL':'механическая',
                                                                 'ROBOT':'роботизированная', 'VARIATOR':'вариатор'})

### 1.2.20. vendor
Не требует преобразований
### 1.2.21. Владельцы

In [ ]:
df_train['Владельцы'] = df_train['Владельцы'].map({4.0:'3 или более', 3.0:'3 или более', 2.0:'2\xa0владельца', 1.0:'1\xa0владелец'})

### 1.2.22. Владение

In [ ]:
def calc_delta_ouned(x):
    if x['year'] > 0:
        now = datetime.now()
        return (now.year - x['year'])*12 - x['month'] + now.month
    else: return 0

df_temp = pd.DataFrame(df_train['Владение'])
pattern = re.compile('\d{4}')
df_temp['year'] = df_temp['Владение'].apply(lambda x:str(pattern.findall(str(x)))[2:-2])
pattern = re.compile('(\d{1,2})\}$')
df_temp['month'] = df_temp['Владение'].apply(lambda x:str(pattern.findall(str(x)))[2:-2])
df_temp['year'] = df_temp['year'].map(lambda x: int(x) if len(x)==4 else 0)
df_temp['month'] = df_temp['month'].map(lambda x: int(x) if len(x)>0 else 0)
df_temp['delta_own'] = df_temp.apply(lambda x: calc_delta_ouned(x), axis=1)
df_train['ownership'] = df_temp['delta_own']

df_temp = pd.DataFrame(df_test['Владение'])
pattern = re.compile('^\d{1,2}')
df_temp['year'] = df_temp['Владение'].apply(lambda x:str(pattern.findall(str(x)))[2:-2])
pattern = re.compile('и\s(\d{1,2})\sм')
df_temp['month'] = df_temp['Владение'].apply(lambda x:str(pattern.findall(str(x)))[2:-2])
df_temp['year'] = df_temp['year'].map(lambda x: int(x) if len(x)>0 else 0)
df_temp['month'] = df_temp['month'].map(lambda x: int(x) if len(x)>0 else 0)
df_temp['delta_own'] = df_temp.apply(lambda x: x['year']*12 + x['month'], axis=1)
df_test['ownership'] = df_temp['delta_own']



### 1.2.23. ПТС

In [ ]:
df_train['ПТС'] = df_train['ПТС'].map({'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})

### 1.2.24. Привод
Не требует преобразований
### 1.2.25. Руль

In [ ]:
df_train['Руль'] = df_train['Руль'].map({'RIGHT':'Правый', 'LEFT':'Левый'})

### 1.2.26. Таможня

In [ ]:
df_train['Таможня'] = df_train['Таможня'].map({True: 'Растаможен', False:'Не растаможен'})

## 1.3. Удаление строк с пропусками

In [ ]:
df_train.dropna(subset=['productionDate','mileage'], inplace=True)
df_train.dropna(subset=['price'], inplace=True)

## 1.4. Объединение "лишних" марок в признаке brand

In [ ]:
# Преобразование привело к снижению результата на 0.03

#target_brand = df_test['brand'].unique().tolist()
#df_train['brand'] = df_train['brand'].map(lambda x: x if x in target_brand else 'OTHER')

## 1.5. Контроль соответствия форматов выборок и объединение

In [ ]:
check_df_before_merg(df_train, df_test)
data = df_test.append(df_train, sort=False).reset_index(drop=True) 
mf.brief_summary(data, [100,75,75,75,75,75,150])

# 2. Дополнение и фильтрация признаков
## 2.1. Исправление признака engineDisplacement
При парсинге тренировочной выборки была допущена ошибка и в признак engineDisplacement попали не корректные значения. Исправим ситуацию, использовав данные из признака name. Также выделим из name информацию об автмобилях с гибридной силовой установкой

In [ ]:
pattern = re.compile('(\d{1}\.\d{1})')
data['engine'] = data['name'].apply(lambda x:str(pattern.findall(str(x)))[2:5])
data['engine'] = pd.to_numeric(data['engine'], errors='coerce')
data['hybrid'] = data['name'].apply(lambda x: 1 if 'hyb' in x else 0)


## 2.2. Тип продавца
Источником косвенной информации о типе продавца является аннотация к объявлению о продаже, сосредоточенная в признаке description. Исползуем проверку по ключевым словам для установления типа продавца.

In [ ]:
data['description'] = data['description'].map(lambda x: str(x).lower())

data['owner'] = data['description'].apply(lambda x: 1 if 'торг' in x or 'не нуждаюсь' in x or len(x)<200 else 0)
data['showroom'] = data['description'].apply(lambda x: 1 if 'traid-in' in x or 'дилер' in x or 'в кредит' in x or 'клиент' in x
                                             or 'без комис' in x or 'страховани' in x or 'трейд-ин' in x 
                                             or 'в наличии' in x or 'выгодное пр' in x or 'автокредит' in x else 0)
display(data['owner'].value_counts())
display(data['showroom'].value_counts())

## 2.3. Период владения

In [ ]:
share = np.around(data['ownership'].value_counts()[0]/len(data)*100, 1)
print(f'Доля пустых значений признака продолжительности владения - {share}%')
# Пока не будем ничего предпринимать в отношении этого признака

## 2.4. Удаление технических столбцов

In [ ]:
drop_cols = ['complectation_dict', 'description', 'equipment_dict', 'engineDisplacement'
             ,'model_info', 'super_gen', 'vehicleConfiguration', 'Владение', 'name'
             ,'priceCurrency', 'Таможня']
data.drop(columns=drop_cols, inplace=True)

## 2.5. Заполнение пропусков

In [ ]:
# Отсутствие количества владельцев указывает на то, что автомобиль новый. Подставляем 0
data['Владельцы'].fillna('Новый', inplace=True)
# Пропуск данных о ПТС заполним значением 'Оригинал'
data['ПТС'].fillna('Оригинал', inplace=True)
# Пропуск данных о рабочем объеме ДВС возникают исключитлеьно у электромобилей. Подставляем 0
data['engine'].fillna(0, inplace=True)
mf.brief_summary(data, [100,75,75,75,75,75,150])

## 2.6. Сортировка признаков по типам

In [ ]:
def sort_features(df_raw, target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col):
    for col in df_raw.columns:
        if col in target_cols or col in time_cols or col in num_cols\
            or col in bin_cols or col in cat_cols or col in count_col:
            continue
        elif len(df_raw[col].value_counts()) == 1:
                df_raw.drop(columns=[col], inplace=True)
        elif at.is_datetime64_any_dtype(df_raw[col]):
            time_cols.append(col)
        elif at.is_numeric_dtype(df_raw[col]):
            if len(df_raw[col].value_counts()) == 2:
                bin_cols.append(col)
            else: num_cols.append(col)
        elif at.is_string_dtype(df_raw[col]):
            cat_cols.append(col)
        else: print(f'Столбец {col} не был причислен ни к одной категории\n'+'_'*50)

    print_cols_lists(df_raw, target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col)

    return target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col

def print_cols_lists(df, target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col):
    print('\nКлючевые признаки: ', target_cols)    
    print('\nПризнаки даты или времени: ', time_cols)    
    print('\nКатегориальные признаки: ', cat_cols)    
    print('\nБинарные признаки: ', bin_cols)    
    print('\nКоличественные признаки: ', num_cols)    
    print('\nПризнаки-счетчики: ', count_col)    

    print('\nВ датасете: строк - ', len(df), 'колонок - ', len(df.columns))


target_cols = ['price']
num_cols, bin_cols, cat_cols, time_cols, count_col = [], [], [], [], []

target_cols,time_cols,num_cols,bin_cols,cat_cols,count_col = sort_features(data
                                                                           ,target_cols,time_cols
                                                                           ,num_cols,bin_cols
                                                                           ,cat_cols,count_col)

## 2.7. Оценка степени влияния признаков на целевые переменные

In [ ]:
for col in cat_cols:
    df_temp = mf.get_label_encoder(data, col, False)
mf.view_important_sign(df_temp, num_cols, bin_cols, cat_cols, 'price')

# 3. Предподготовка данных
## 3.1. Стандартизация

In [ ]:
data = mf.normalisation(data, MinMaxScaler(), target_cols + cat_cols)
display(data.head(1))

## 3.2. Label Encoding

In [ ]:
for colum in cat_cols:
    data[colum] = data[colum].astype('category').cat.codes
display(data.head(1))

## 3.3. Генерируем полиномиальные признаки

In [ ]:
# Результат применения неудовлетворительный. Рост MAPE при тестировании на 1%

#pf = PolynomialFeatures(2)
#poly_df = pf.fit_transform(data[num_cols])
#poly_df = pd.DataFrame(poly_df)
#poly_df = poly_df.drop(0, axis=1)
#data = pd.concat([data, poly_df],axis=1)
#data

## 3.4. Поэлементный контроль

In [ ]:
pd.set_option('display.max_columns', None)
display(data.sample(3).head(1))
display(data.describe())

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price'], axis=1)
y = data.query('sample == 1')['price'] 
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)

## 3.5. Разделение датасета

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# 4. Модели

In [ ]:
def print_learn_report(start, y_test, y_pred):
    print('\nВремя выполнения - ', datetime.now() - start)
    print(f"Точность по метрике MAPE:{(mape(y_test, y_pred))*100:0.2f}%")

## 4.1. Наивная модель
Эта модель будет предсказывать среднюю цену по рабочему объему двигателя. 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engine'].map(tmp_train.groupby('engine')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

## 4.2. CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


### 4.2.1. Линейные данные

In [ ]:
start = datetime.now()

model = CatBoostRegressor(iterations = 50,
                          cat_features=cat_cols,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         use_best_model=True,
         verbose=False,
         plot=True
         )

model.save_model('catboost_model.model')

predict = model.predict(X_test)
print_learn_report(start, y_test, predict)

Для 5000 итераций:
Время выполнения -  0:09:43.214411
Точность по метрике MAPE:12.89%
### 4.2.2. Логарифмирование целевой переменной и подбор наилучших параметров
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    

In [ ]:
start = datetime.now()



model = CatBoostRegressor(cat_features=cat_cols, iterations=50
                          ,loss_function='MAPE', metric_period=10)

grid = {'learning_rate': [ 0.13, 0.14, 0.15]
        ,'depth': [12]
        ,'l2_leaf_reg': [7, 7.5, 8]
        ,'random_strength': [0.3]}

#grid_search_result = model.grid_search(grid
#                                     ,X=X_train
#                                     ,y=np.log(y_train)
#                                     ,plot=True
#                                     ,verbose=False)
#print('\nВремя выполнения - ', datetime.now() - start)
#print('\nНаилучшие параметры: ', grid_search_result['params'])


cb = CatBoostRegressor(iterations = 5000,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'],
                       silent=True,
                       learning_rate=0.13, depth=12,
                       l2_leaf_reg=8, random_strength=0.3)

#cb.fit(X_train, np.log(y_train),
#         eval_set=(X_test, np.log(y_test)),
#         verbose=False,
#         use_best_model=True,
#         plot=True)

#cb.save_model('catboost_log_model.model')

#predict_test = np.exp(cb.predict(X_test))
#print_learn_report(start, y_test, predict_test)

Время выполнения -  0:08:29.342993

Точность по метрике MAPE:10.88%

## 4.3. Random Forest
Определим лучшие параметры и обучим на них модель

In [ ]:
start = datetime.now()

random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

#rfr = RandomForestRegressor(random_state = RANDOM_SEED)
#rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=RANDOM_SEED, n_jobs = -1)
#rf_random.fit(X_train, np.log(y_train))
#rf_random.best_params_

#best_params_: 
#{'n_estimators': 300,
# 'min_samples_split': 2,
# 'min_samples_leaf': 1,
# 'max_features': 'sqrt',
# 'max_depth': None,
# 'bootstrap': False}

#best_rfr = rf_random.best_estimator_
best_rfr = RandomForestRegressor(random_state=RANDOM_SEED
                      , n_estimators=300
                      , min_samples_split=2
                      , min_samples_leaf=1
                      , max_features='sqrt'
                      , max_depth=None
                      , bootstrap=False)

#best_rfr.fit(X_train, np.log(y_train))


#predict_rfr = np.exp(best_rfr.predict(X_test))
#print_learn_report(start, y_test, predict_rfr)

Время выполнения -  0:00:52.179295

Точность по метрике MAPE:10.96%

## 4.4. Gradient Boosting Regressor

In [ ]:
start = datetime.now()

random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 8)],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4]}

#gbr = GradientBoostingRegressor()
#gbr_random = RandomizedSearchCV(estimator = gbr, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=RANDOM_SEED, n_jobs = -1)
#gbr_random.fit(X_train, np.log(y_train))
#gbr_random.best_params_
best_gbr = GradientBoostingRegressor(random_state=RANDOM_SEED
                      , n_estimators=800
                      , min_samples_split=5
                      , min_samples_leaf=4
                      , max_features='sqrt'
                      , max_depth=9)
#best_gbr.fit(X_train, np.log(y_train))

#predict_gbr = np.exp(best_gbr.predict(X_test))
#print_learn_report(start, y_test, predict_gbr)

Время выполнения -  0:01:03.301634

Точность по метрике MAPE:10.87%

## 4.5. XGB Regressor

In [ ]:
start = datetime.now()

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate=0.05, max_depth=12, alpha=1,
                          n_estimators=1000)
#xgb_reg.fit(X_train, np.log(y_train))
#xgb_red_pred = np.exp(xgb_reg.predict(X_test))
#print_learn_report(start, y_test, xgb_red_pred)

Время выполнения -  0:00:58.151827

Точность по метрике MAPE:10.53%

# 5. Бэггинг
## 5.1. Со случайным лесом

In [ ]:
start = datetime.now()

bagg_rfr = BaggingRegressor(best_rfr, n_estimators=3, n_jobs=1, random_state=RANDOM_SEED)
#bagg_rfr.fit(X_train, np.log(y_train))
#predict_bagg_rfr = np.exp(bagg_rfr.predict(X_test))
#print_learn_report(start, y_test, predict_bagg_rfr)

Время выполнения -  0:01:35.995616

Точность по метрике MAPE:11.24%

## 5.2. С градиентным бустингом

In [ ]:
start = datetime.now()

bagg_gbr = BaggingRegressor(best_gbr, n_estimators=3, n_jobs=1, random_state=RANDOM_SEED)
#bagg_gbr.fit(X_train, np.log(y_train))
#predict_bagg_gbr = np.exp(bagg_gbr.predict(X_test))
#print_learn_report(start, y_test, predict_bagg_gbr)

Время выполнения -  0:01:47.575827

Точность по метрике MAPE:10.73%

## 5.3. С xgb

In [ ]:
start = datetime.now()

bagg_xgb = BaggingRegressor(xgb_reg, n_estimators=3, n_jobs=1, random_state=RANDOM_SEED)
#bagg_xgb.fit(X_train, np.log(y_train))
#predict_bagg_xgb = np.exp(bagg_xgb.predict(X_test))
#print_learn_report(start, y_test, predict_bagg_xgb)

Время выполнения -  0:02:35.895540

Точность по метрике MAPE:10.62%

В результате применения бэггинга улучшены результаты для Gradient Boosting Regressor. Результаты Random Forest Regressor и XGB Regressor снизились.

# 6. Стеккинг. Ансамбли моделей

In [ ]:
start = datetime.now()

estimators=[('b_gbr', BaggingRegressor(GradientBoostingRegressor(random_state=RANDOM_SEED
                                                                ,n_estimators=800
                                                                ,min_samples_split=5
                                                                ,min_samples_leaf=4
                                                                ,max_features='sqrt'
                                                                ,max_depth=9)
                                        ,n_estimators=3
                                        ,n_jobs=1
                                        ,random_state=RANDOM_SEED))
            ,('xgb', xgb.XGBRegressor(objective='reg:squarederror'
                                      ,colsample_bytree=0.5
                                      ,learning_rate=0.05
                                      ,max_depth=12
                                      ,alpha=1
                                      ,n_estimators=1000))]

st_ensemble = StackingRegressor(estimators=estimators
                                ,final_estimator = CatBoostRegressor(iterations = 5000
                                                                     ,random_seed = RANDOM_SEED
                                                                     ,eval_metric='MAPE'
                                                                     ,custom_metric=['R2', 'MAE']
                                                                     ,silent=True
                                                                     ,learning_rate=0.13
                                                                     ,depth=12
                                                                     ,l2_leaf_reg=8
                                                                     ,random_strength=0.3))

    
st_ensemble.fit(X_train, np.log(y_train))

predict_ensemble = np.exp(st_ensemble.predict(X_test))
print_learn_report(start, y_test, predict_ensemble)

# 7. Submission

In [ ]:
st_ensemble.fit(X, np.log(y))

predict_submission = np.round(np.exp(st_ensemble.predict(X_sub)),-3)
predict_submission 

In [ ]:
sample_submission['price'] = predict_submission * k
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

# What's next?
Что еще можно сделать, чтобы улучшить результат:

* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Попробовать другие алгоритмы и библиотеки ML
* Использование Blending & Stacking не принесло ожидаемого улучшеня качества предсказани. Наилучший результат имело применение Random Forest с регуляризацией.

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing